In [4]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import hashlib
import time
import copy

from numpy.linalg import matrix_rank
from tqdm import tqdm

In [5]:
# def get_G(alphas, m, d):
#     poly = [0] * (d)

#     for seq in generate(d - 1):
#         ones = np.count_nonzero(seq)
#         zeros = d - 1 - ones
        
#         power = 0
#         for j in range(len(seq)):
#             if seq[j] == 1:
#                 power = (power + (j + 1)) % (2**m - 1)

#         poly[zeros] = (poly[zeros] + alphas[power]) % 2
        
#     return poly
    
# def print_G(g, alphas):
#     p = ''
#     for i in range(len(g)):
#         id = -1
#         for j in range(len(alphas)):
#             if np.all(np.equal(g[i], alphas[j])):
#                 id = j
#                 break
#         p = p + 'alpha^{} * x^{} + '.format(id, i)
#     print(p[:-2])

# g = get_G(alphas, m, d)
# print_G(g, alphas)

In [3]:
def invert(v, i):
    v[i] = (v[i] + 1) % 2

# c = np.zeros(2**m - 1)
# temp = np.polymul(g, [1, 0, 0])
# for i in range(1, len(temp) + 1):
#     c[-i] = temp[-i]

# v = np.array(c[::-1])
# invert(v, 0)
# invert(v, 1)
# v = v[::-1]

# print('c = {}'.format(c))
# print('v = {}'.format(v))

In [6]:
s = np.polydiv(v, g)[1] % 2 # остаток от деления входа на порождающий полином 
print(polynomial_to_string(s[::-1]))

NameError: name 'v' is not defined

In [7]:
def get_roots_3(p, m):
    power = 2 ** m - 1;

    # transform to make first element 0
    add = power - p[0]
    for i in range(len(p)):
        p[i] = (p[i] + add) % power
    
    for x in range(power):
        for y in range(power):
            for z in range(power):
                pr = False
#                 if (x == 0 and y == 1 and z == 2):
#                     pr = True
                    
#                 if pr:
#                     print((x+y+z)%power)
                if (x + y + z) % power != p[3]:
                    continue
                sum2 = (alphas[(x + y) % power] + alphas[(x + z) % power] + alphas[(y + z) % power]) % 2
                k2 = locate(sum2, alphas)
                if (k2 != p[2]):
                    continue
                sum3 = (alphas[x] + alphas[y] + alphas[z]) % 2
                k3 = locate(sum3, alphas) 
#                 print('k3', k3)
                if (k3 != p[1]):
                    continue
                return [x, y, z]
    return [-1, -1, -1]


In [8]:
def get_solution_3(matrix, result, power):
    n = len(matrix)
    m = len(matrix[0])
    mod = (2 ** power) - 1
    
    for x in range(2**power):
        for y in range(2**power):
            for z in range(2**power):
        
                result_check = []
                check = [x, y, z]
                for i in range(n):
                    s = np.zeros(power)
                    for j in range(m):
                        k = (matrix[i][j] + check[j]) % mod
                        s += alphas[k]
                    s %= 2
                    result_check.append(locate(s, alphas))
                if (np.all(result_check == result)):
#                     print(result_check)
                    return check
        
    return [-1, -1, -1]

In [9]:

def get_lambdas_3(S):
    return get_solution_3(
        [
            [S[0], S[1], S[2]], 
            [S[1], S[2], S[3]], 
            [S[2], S[3], S[4]]
        ], 
        [S[3], S[4], S[5]],
        5
    )

In [10]:
def solve_3(v, start, d):
#     print('v = {}'.format(v))
    S, nS = get_syndrom_components(v, start, d)
    print('S = {}'.format(nS))
    lambdas = get_lambdas_3(nS)
    print('lambdas = {}'.format(lambdas))
    
    lambda_polynomial = np.zeros(4)
    lambda_polynomial[-1] = 0
    lambda_polynomial[-2] = lambdas[2]
    lambda_polynomial[-3] = lambdas[1]
    lambda_polynomial[-4] = lambdas[0]
#     print('lambda poly = {}'.format(lambda_polynomial))
    
    lambda_roots = get_roots_3(lambda_polynomial, m)
    lambda_roots = [(31 - root) % 31 for root in lambda_roots]
    print('error positions = {}'.format(lambda_roots))
    return lambda_roots